In [35]:
import pandas as pd 
import numpy as np
import re

### Connecting runs with sampels 
In order to connect sample Ids with run ids the only key I had was the number of reads. This notebook contains some steps regarding this process:

1) 
the read count I had was forward and reverse combined so I divided this into two to avoid counting both forward and reverse reads. And saved it as a csv file to use as input in the script

2) 
Then I made the script count_read_to_match_id.py - The output was not as hoped since none of the read counts matched any of the counts I had been given - but here is the list of the acctual counts in the reads. 

###  1) Dividing the reads in two

In [36]:
NZ_reads = pd.read_csv('3regionData/Sample_Rawreads_NZ.tsv', sep = '\t')

In [37]:
NZ_reads['#Rawreads'] = NZ_reads['#Rawreads'].astype(int)

In [38]:
NZ_reads['#Rawreads']

0      265275932
1      261070644
2      259086484
3      246402572
4      244068892
         ...    
316    102916696
317     99312498
318     98078308
319     96545846
320     42722198
Name: #Rawreads, Length: 321, dtype: int64

In [39]:
NZ_reads['#Rawreads'] = NZ_reads['#Rawreads']/2

In [40]:
NZ_reads['#Rawreads'] = NZ_reads['#Rawreads'].astype(int)

In [41]:
NZ_reads

,Sample,#Rawreads
0,F1606-058,132637966
1,F1604-045,130535322
2,F1604-007,129543242
3,F1605-009,123201286
4,F1604-022,122034446
...,...,...
316,F1513-219,51458348
317,F1602_059,49656249
318,F1604_039,49039154
319,F1608_001_3,48272923


In [26]:
NZ_reads.to_csv('NZ_samples_1read_ID.csv', index=False)

### 2) cleaning output from script saving the read counts

In [59]:
script_output = pd.read_csv('3regionData/33773072.qsub_read_match_id.stdout', header = None)

In [60]:
script_output

,0
0,CL100023189_L02_38_1.fq.gz
1,CL100006258_L01_7_1.fq.gz
2,CL100017972_L02_11_1.fq.gz
3,CL100006258_L01_4_1.fq.gz
4,CL100017973_L01_1_1.fq.gz
...,...
634,82069540 was not present as read count in the...
635,82787491 was not present as read count in the...
636,81837025 was not present as read count in the...
637,59731009 was not present as read count in the...


In [81]:
files = script_output.loc[script_output[0].str.startswith('CL', na=False)]

In [84]:
reads = script_output.iloc[319:]

In [85]:
reads

,0
319,89082404
320,66217371
321,88436484
322,53064542
323,91208499
...,...
634,82069540
635,82787491
636,81837025
637,59731009


In [64]:
reads[0] = reads[0].str.extract('(\d+)', expand=False)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [86]:
reads= reads.rename({0: 'read_count'}, axis=1)

In [87]:
files = files.rename({0: 'run_id'}, axis=1)

In [88]:
files

,run_id
0,CL100023189_L02_38_1.fq.gz
1,CL100006258_L01_7_1.fq.gz
2,CL100017972_L02_11_1.fq.gz
3,CL100006258_L01_4_1.fq.gz
4,CL100017973_L01_1_1.fq.gz
...,...
314,CL100020401_L02_10_1.fq.gz
315,CL100017972_L02_12_1.fq.gz
316,CL100020404_L01_1_1.fq.gz
317,CL100006258_L02_13_1.fq.gz


In [93]:
reads['read_count'] = reads['read_count'].astype(int)

In [102]:
reads['read_count'].max()

134677219

In [106]:
#removing the value 0 which was in there. 
reads = reads[reads.read_count != 0]

In [108]:
reads.min()

read_count    16998916
dtype: int64

In [123]:
output = dict(zip(files['run_id'], reads['read_count']))

In [124]:
output

{'CL100023189_L02_38_1.fq.gz': 89082404,
 'CL100006258_L01_7_1.fq.gz': 66217371,
 'CL100017972_L02_11_1.fq.gz': 88436484,
 'CL100006258_L01_4_1.fq.gz': 53064542,
 'CL100017973_L01_1_1.fq.gz': 91208499,
 'CL100020401_L02_9_1.fq.gz': 103305735,
 'CL100020243_L01_21_1.fq.gz': 85218066,
 'CL100013689_L01_16_1.fq.gz': 86089720,
 'CL100020401_L02_13_1.fq.gz': 72821190,
 'CL100020425_L02_40_1.fq.gz': 63909128,
 'CL100013751_L02_4_1.fq.gz': 81692793,
 'CL100017863_L02_11_1.fq.gz': 82309896,
 'CL100020456_L02_35_1.fq.gz': 92152731,
 'CL100006258_L02_12_1.fq.gz': 50373388,
 'CL100006210_L02_14_1.fq.gz': 64511983,
 'CL100013749_L01_10_1.fq.gz': 66936795,
 'CL100020244_L01_3_1.fq.gz': 53883929,
 'CL100013751_L02_1_1.fq.gz': 81976836,
 'CL100006375_L02_16_1.fq.gz': 82851543,
 'CL100013751_L01_16_1.fq.gz': 98988795,
 'CL100006258_L01_5_1.fq.gz': 58350228,
 'CL100020242_L02_12_1.fq.gz': 72501157,
 'CL100017975_L01_2_1.fq.gz': 81153810,
 'CL100013751_L02_5_1.fq.gz': 80694470,
 'CL100017971_L02_9_1.fq.

294016368/4 = 73504092.0 is the number of reads in CL100013689_L01_15_1.fq.gz - just to check if it fits with the row below since there is 0 in the middle of the reads and a read count more than there is files!

With another method i counted 107 of the files: 

In [293]:
CountMethod2 = pd.read_csv('3regionData/CL_files_read_count.stdout', sep = ' ', header=None)

In [294]:
CountMethod2 = CountMethod2.rename({0: 'line_count', 1: 'filepath'}, axis=1)

In [295]:
CountMethod2

,line_count,filepath
0,269026252,CL100006210_L01_1_1/CL100006210_L01_1_1.fq.gz
1,249986008,CL100006210_L01_2_1/CL100006210_L01_2_1.fq.gz
2,250339448,CL100006210_L01_3_1/CL100006210_L01_3_1.fq.gz
3,257594976,CL100006210_L01_4_1/CL100006210_L01_4_1.fq.gz
4,328774404,CL100006210_L01_5_1/CL100006210_L01_5_1.fq.gz
...,...,...
314,257947912,CL100023189_L02_36_1/CL100023189_L02_36_1.fq.gz
315,234373640,CL100023189_L02_37_1/CL100023189_L02_37_1.fq.gz
316,350867368,CL100023189_L02_38_1/CL100023189_L02_38_1.fq.gz
317,381387812,CL100023189_L02_39_1/CL100023189_L02_39_1.fq.gz


In [296]:
CountMethod2['read_count'] = (CountMethod2['line_count']/4).astype(int)

In [297]:
CountMethod2['run_id'] = CountMethod2['filepath'].str.split('/', expand=True)[1]

In [298]:
CountMethod2

,line_count,filepath,read_count,run_id
0,269026252,CL100006210_L01_1_1/CL100006210_L01_1_1.fq.gz,67256563,CL100006210_L01_1_1.fq.gz
1,249986008,CL100006210_L01_2_1/CL100006210_L01_2_1.fq.gz,62496502,CL100006210_L01_2_1.fq.gz
2,250339448,CL100006210_L01_3_1/CL100006210_L01_3_1.fq.gz,62584862,CL100006210_L01_3_1.fq.gz
3,257594976,CL100006210_L01_4_1/CL100006210_L01_4_1.fq.gz,64398744,CL100006210_L01_4_1.fq.gz
4,328774404,CL100006210_L01_5_1/CL100006210_L01_5_1.fq.gz,82193601,CL100006210_L01_5_1.fq.gz
...,...,...,...,...
314,257947912,CL100023189_L02_36_1/CL100023189_L02_36_1.fq.gz,64486978,CL100023189_L02_36_1.fq.gz
315,234373640,CL100023189_L02_37_1/CL100023189_L02_37_1.fq.gz,58593410,CL100023189_L02_37_1.fq.gz
316,350867368,CL100023189_L02_38_1/CL100023189_L02_38_1.fq.gz,87716842,CL100023189_L02_38_1.fq.gz
317,381387812,CL100023189_L02_39_1/CL100023189_L02_39_1.fq.gz,95346953,CL100023189_L02_39_1.fq.gz


In [299]:
CountMethod2_dict = dict(zip(CountMethod2['run_id'], CountMethod2['read_count']))

In [301]:
for key in CountMethod2_dict:
    if not key in output:
        continue
    print('Count_output', output[key])
    print('CountMethod2', CountMethod2_dict[key])
    print('difference:',output[key]-CountMethod2_dict[key])

Count_output 68157182
CountMethod2 67256563
difference: 900619
Count_output 63325973
CountMethod2 62496502
difference: 829471
Count_output 63416656
CountMethod2 62584862
difference: 831794
Count_output 65264742
CountMethod2 64398744
difference: 865998
Count_output 83297831
CountMethod2 82193601
difference: 1104230
Count_output 82098018
CountMethod2 81003026
difference: 1094992
Count_output 76355522
CountMethod2 75346943
difference: 1008579
Count_output 70469654
CountMethod2 69540530
difference: 929124
Count_output 66084826
CountMethod2 65179627
difference: 905199
Count_output 68700247
CountMethod2 67759374
difference: 940873
Count_output 56526207
CountMethod2 55747891
difference: 778316
Count_output 53583139
CountMethod2 52842620
difference: 740519
Count_output 64511983
CountMethod2 63614106
difference: 897877
Count_output 58883901
CountMethod2 58076757
difference: 807144
Count_output 62235748
CountMethod2 61389838
difference: 845910
Count_output 65633752
CountMethod2 64743018
differen

CountMethod2 111400205
difference: 1669320
Count_output 104651408
CountMethod2 103099179
difference: 1552229
Count_output 82069540
CountMethod2 81027337
difference: 1042203
Count_output 81174150
CountMethod2 80138996
difference: 1035154
Count_output 63878103
CountMethod2 63052184
difference: 825919
Count_output 72821190
CountMethod2 71896488
difference: 924702
Count_output 75241827
CountMethod2 74306499
difference: 935328
Count_output 76966439
CountMethod2 75997916
difference: 968523
Count_output 97497930
CountMethod2 96262446
difference: 1235484
Count_output 103305735
CountMethod2 101948351
difference: 1357384
Count_output 81837025
CountMethod2 80574372
difference: 1262653
Count_output 75544894
CountMethod2 74352388
difference: 1192506
Count_output 107450609
CountMethod2 105834859
difference: 1615750
Count_output 100257564
CountMethod2 98771882
difference: 1485682
Count_output 90159575
CountMethod2 88818427
difference: 1341148
Count_output 67962007
CountMethod2 66933403
difference: 10

So it looks like the countMethod2 is alwas a little smaller than the output from the script. But do any of them match the information given to fit with the sample id?


In [302]:
original_info = pd.read_csv('3regionData/original_reads_information.tsv', sep='\t')

In [303]:
original_info

,Sample,#Rawreads,Readslength(bp),#HQpaid-endreads1,#HQpaid-endreads2,#HQreads,HQratio,#HQrmhostpair-endreads,#HQrmhostsingle-endreads,#HQrmhostreads
0,F1606-058,265275932,50,131599927,131804460,263404387,"99,29%",262172374,1112195,263284569
1,F1604-045,261070644,50,129643702,128649729,258293431,"98,94%",247631872,1975945,249607817
2,F1604-007,259086484,50,129014882,127202704,256217586,"98,89%",251849712,2108939,253958651
3,F1605-009,246402572,50,122496781,121421860,243918641,"98,99%",241657802,1929195,243586997
4,F1604-022,244068892,50,119005624,120093489,239099113,"97,96%",234821120,3275255,238096375
...,...,...,...,...,...,...,...,...,...,...
316,F1513-219,102916696,50,51099361,50050219,101149580,"98,28%",98091930,1305042,99396972
317,F1602_059,99312498,50,48707098,47488125,96195223,"96,86%",92961620,2656802,95618422
318,F1604_039,98078308,50,48121477,46798300,94919777,"96,78%",91685194,2711398,94396592
319,F1608_001_3,96545846,50,47803574,45679067,93482641,"96,83%",89311980,2670124,91982104


In [304]:
def matching_dict_with_orignal(reads_dict, original_reads_step):
    for read in (original_info[original_reads_step]/2).astype(int):
        for key in reads_dict:
            if reads_dict[key] == read:
                print('there was a match for', key,'with reads count',read)
            

In [305]:
matching_dict_with_orignal(output, '#Rawreads')

In [306]:
matching_dict_with_orignal(output, '#HQreads')

In [307]:
matching_dict_with_orignal(output, '#HQrmhostreads')

In [308]:
matching_dict_with_orignal(CountMethod2_dict, '#Rawreads')

there was a match for CL100023189_L02_35_1.fq.gz with reads count 132637966
there was a match for CL100023189_L01_23_1.fq.gz with reads count 130535322
there was a match for CL100017968_L01_2_1.fq.gz with reads count 129543242
there was a match for CL100020565_L02_16_1.fq.gz with reads count 123201286
there was a match for CL100020404_L01_7_1.fq.gz with reads count 122034446
there was a match for CL100020242_L02_16_1.fq.gz with reads count 120616851
there was a match for CL100017971_L01_7_1.fq.gz with reads count 120486254
there was a match for CL100017968_L02_10_1.fq.gz with reads count 117762934
there was a match for CL100023189_L01_21_1.fq.gz with reads count 117005415
there was a match for CL100020456_L02_39_1.fq.gz with reads count 113763987
there was a match for CL100020401_L01_7_1.fq.gz with reads count 111400205
there was a match for CL100017971_L02_13_1.fq.gz with reads count 110353625
there was a match for CL100017971_L02_16_1.fq.gz with reads count 109653891
there was a matc

The counts received using the command: for x in CL100*/*_1.fq.gz; do unpigz -p 8 -c $x | wc -l && echo $x; done
gave us counts that matches the raw reads.

creating a dict to connect sample and run ids. 

In [309]:
sample_dict = dict(zip((original_info['#Rawreads']/2).astype(int), original_info['Sample']))

In [310]:
sample_dict

{132637966: 'F1606-058',
 130535322: 'F1604-045',
 129543242: 'F1604-007',
 123201286: 'F1605-009',
 122034446: 'F1604-022',
 120616851: 'F1513-133',
 120486254: 'F1606-023',
 117762934: 'F1604-009',
 117005415: 'F1604-042',
 113763987: 'F1602-078',
 111842531: 'F1602-081',
 111400205: 'F1604-032',
 110353625: 'F1606-031',
 109653891: 'F1602-076',
 109549289: 'F1604-008',
 108252248: 'F1513-080',
 105834859: 'F1602-118',
 104453935: 'F1513-039',
 104423695: 'F1606-033',
 103904714: 'F1602_007',
 103815173: 'F1608-022',
 103643611: 'F1604-015',
 103579359: 'F1605-001',
 103099179: 'F1604-036',
 101948351: 'F1606-045',
 101275103: 'F1513-070',
 100597886: 'F1513-130',
 100133592: 'F1606_024',
 99861406: 'F1606_019',
 98771882: 'F1608-024',
 98728656: 'F1605-008',
 98276715: 'F1513-194',
 98196243: 'F1513-140',
 97884938: 'F1606-029',
 97861926: 'F1513_069',
 97841735: 'F1513-179',
 97540979: 'F1604-001',
 96262446: 'F1602-092',
 95741762: 'F1605-012',
 95346953: 'F1605-043',
 94802359: '

In [311]:
def combine_readcount_with_ID(ID_reads, folder_reads):
    unmatched_samples = []
    for key, value in folder_reads.items():
        if value in ID_reads:
            ID_reads[value] = [ID_reads[value], key]
        else:
            print(value, ' was not present as read count in the samples list')
            unmatched_samples.append(key)
    return ID_reads, unmatched_samples

In [312]:
combined_ids, unmatched_samples = combine_readcount_with_ID(sample_dict, CountMethod2_dict)

16295420  was not present as read count in the samples list


In [313]:
unmatched_samples

['CL100020243_L02_37_1.fq.gz']

In [316]:
combined_ids

{132637966: ['F1606-058', 'CL100023189_L02_35_1.fq.gz'],
 130535322: ['F1604-045', 'CL100023189_L01_23_1.fq.gz'],
 129543242: ['F1604-007', 'CL100017968_L01_2_1.fq.gz'],
 123201286: ['F1605-009', 'CL100020565_L02_16_1.fq.gz'],
 122034446: ['F1604-022', 'CL100020404_L01_7_1.fq.gz'],
 120616851: ['F1513-133', 'CL100020242_L02_16_1.fq.gz'],
 120486254: ['F1606-023', 'CL100017971_L01_7_1.fq.gz'],
 117762934: ['F1604-009', 'CL100017968_L02_10_1.fq.gz'],
 117005415: ['F1604-042', 'CL100023189_L01_21_1.fq.gz'],
 113763987: ['F1602-078', 'CL100020456_L02_39_1.fq.gz'],
 111842531: 'F1602-081',
 111400205: ['F1604-032', 'CL100020401_L01_7_1.fq.gz'],
 110353625: ['F1606-031', 'CL100017971_L02_13_1.fq.gz'],
 109653891: ['F1602-076', 'CL100017971_L02_16_1.fq.gz'],
 109549289: ['F1604-008', 'CL100020565_L01_4_1.fq.gz'],
 108252248: ['F1513-080', 'CL100020243_L02_39_1.fq.gz'],
 105834859: ['F1602-118', 'CL100020404_L01_3_1.fq.gz'],
 104453935: ['F1513-039', 'CL100017973_L02_15_1.fq.gz'],
 104423695: 

In [318]:
ids_df = pd.DataFrame.from_dict(combined_ids, orient='index')
ids_df

,0,1,2,3,4,5,6,7,8
132637966,F1606-058,CL100023189_L02_35_1.fq.gz,None,None,None,None,None,None,None
130535322,F1604-045,CL100023189_L01_23_1.fq.gz,None,None,None,None,None,None,None
129543242,F1604-007,CL100017968_L01_2_1.fq.gz,None,None,None,None,None,None,None
123201286,F1605-009,CL100020565_L02_16_1.fq.gz,None,None,None,None,None,None,None
122034446,F1604-022,CL100020404_L01_7_1.fq.gz,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
51458348,F1513-219,CL100020244_L01_4_1.fq.gz,None,None,None,None,None,None,None
49656249,F1602_059,CL100006258_L02_12_1.fq.gz,None,None,None,None,None,None,None
49039154,F1604_039,CL100006258_L02_14_1.fq.gz,None,None,None,None,None,None,None
48272923,F1608_001_3,CL100006258_L01_3_1.fq.gz,None,None,None,None,None,None,None


In [329]:
id_translation_df = pd.DataFrame(ids_df).rename({0:'Sample_id', 1:'run_id'}, axis=1)

In [337]:
id_translation_df.iloc[:,[0,1]].to_csv('3regionData/run_id_sample_id_translater.csv', index=True)

In [335]:
len(id_translation_df)

321

In [334]:
id_translation_df.iloc[:,[0,1]]

,Sample_id,run_id
132637966,F1606-058,CL100023189_L02_35_1.fq.gz
130535322,F1604-045,CL100023189_L01_23_1.fq.gz
129543242,F1604-007,CL100017968_L01_2_1.fq.gz
123201286,F1605-009,CL100020565_L02_16_1.fq.gz
122034446,F1604-022,CL100020404_L01_7_1.fq.gz
...,...,...
51458348,F1513-219,CL100020244_L01_4_1.fq.gz
49656249,F1602_059,CL100006258_L02_12_1.fq.gz
49039154,F1604_039,CL100006258_L02_14_1.fq.gz
48272923,F1608_001_3,CL100006258_L01_3_1.fq.gz
